In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
def generate_linear_regression_data(m: int, n: int, mean=0, std=1, noise_std=0.5):
    """
    Generates synthetic data for linear regression.

    Parameters:
    - m: Number of samples.
    - n: Number of features per sample.
    - noise_std (float): Standard deviation of the Gaussian noise added to y.

    Returns:
    - X (np.ndarray): Feature matrix of shape (m, n).
    - y (np.ndarray): Target vector of shape (m,).
    """

    # Generate feature matrix X from a standard normal distribution
    X = np.random.normal(mean, std, (m, n))

    # Generate true coefficients (weights) and intercept (bias)
    w = np.random.randn(n, 1)  # true coefficients
    b = np.random.randn()      # intercept term

    y = X.dot(w) + b

    # Add Gaussian noise to the target values
    y += np.random.normal(0, noise_std, (m, 1))

    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=SEED)

    # Sort
    idx = np.argsort(X_train[:, 0])
    X_train = X_train[idx, :]
    y_train = y_train[idx]

    return X_train, X_test, y_train, y_test

In [3]:
# From LinearRegressionFL_v1.ipynb notebook
def generate_weight_matrix(n):
    W = np.random.rand(n+1, n+2).astype(np.float32)
    W = np.concatenate([W, np.ones((1, n+2))], axis=0)

    diag = np.sum(np.abs(W), axis=0) + 1
    np.fill_diagonal(W, diag)
    W[n+1, n+1] = 1         # the row of 1's was also affected, so reassigning a value of 1 again.

    return W

def client_calculate_gradients(W, X, y):
    batch_size = X.shape[0]

    # Add an intercept column
    X = np.hstack([np.ones((batch_size, 1)), X])

    # Remove the last row of one's from W
    W = W[:-1, :]

    # Make Y matrix out of n+2 copies of y to count for n+2 random sets of weights
    Y = np.hstack([y]*(n+2))

    # Calculate the gradient dL/dw
    L = X.T@(X @ W - Y)# * (1/batch_size)

    return L

In [4]:
SEED = 43
np.random.seed(SEED)

In [5]:
n = 7
CLIENT_NUM = 5

DATA_SIZE = np.random.randint(1000, 10000, size=CLIENT_NUM)
MEAN = np.random.randint(10, 20, size=CLIENT_NUM)
STD = np.random.randint(5, 15, size=CLIENT_NUM)
NOISE_STD = np.random.uniform(2, 5, size=CLIENT_NUM)

In [6]:
data = []       # [(X_train, X_test, y_train, y_test), (X_train, X_test, y_train, y_test), ...]
for i in range(CLIENT_NUM):
    data.append(generate_linear_regression_data(DATA_SIZE[i], n, MEAN[i], STD[i], NOISE_STD[i]))

In [7]:
# Generate weight matrix W
W = generate_weight_matrix(n)
# Check its determinant
np.linalg.det(W)

np.float64(559108.8079447665)

In [8]:
# Compute gradients for each client (forward pass)
gradients = []
BETA = np.random.uniform(-0.1, 0.1, size=CLIENT_NUM)    # beta parameter for rescaling the gradient matrices
for i in range(CLIENT_NUM):
    X_train = data[i][0]
    y_train = data[i][2]
    gradients.append((1-BETA[i])*client_calculate_gradients(W, X_train, y_train))

In [9]:
L = np.sum(gradients, axis=0)

# Calculate w_opt by the algorithm 1
C = L @ np.linalg.inv(W)
A = C[:, :-1]
b = C[:, -1] * -1
w_opt = np.linalg.solve(A, b)    # [w0, w1, w2, ...]

X_test = data[0][1]
y_test = data[0][3]
for i in range(1, CLIENT_NUM):
    X_test = np.concatenate((X_test, data[i][1]), axis=0)
    y_test = np.concatenate((y_test, data[i][3]), axis=0)

# Predict
X_test = np.hstack([np.ones((X_test.shape[0], 1)), X_test])
y_pred = X_test @ w_opt

In [10]:
# Table must cover the results of mse with the following variables
# noise_std: low, medium, high,
# row: v1, v2
# BETA ranges: (-0.1, 0.1); (-0.05, 0.05); (-0.01, 0.01)

In [12]:
# Print the results
print("CLIENT_DATA_SIZE:", DATA_SIZE)
print("CLIENT_MEAN:", MEAN)
print("CLIENT_STD:", STD)
print("CLIENT_NOISE_STD:", NOISE_STD)
print()
print("BETA:", BETA)
print()
print("Overall MSE:", mean_squared_error(y_test, y_pred))
print("Overall R²:", r2_score(y_test, y_pred))

CLIENT_DATA_SIZE: [4392 3303 8985 3064 9499]
CLIENT_MEAN: [11 12 17 10 13]
CLIENT_STD: [ 7 14  6  7  7]
CLIENT_NOISE_STD: [4.86422461 3.6673356  3.94916503 3.7426317  2.50664548]

BETA: [-0.04155807  0.08045028  0.09525173 -0.02374181  0.02721002]

Overall MSE: 358.2773939989091
Overall R²: 0.07914599641953313
